In [9]:
import requests
import pandas as pd
import json
from dateutil import parser

In [11]:

session=requests.Session()

In [12]:
session.headers.update()

url=f"{OANDA_URL}/instruments/EUR_USD/candles"
url_inst=f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

params=dict(count=10,granularity="H1", price="MBA")

In [13]:
response=session.get(url_inst,params=None,data=None, headers=None)

In [14]:
response.status_code

200

In [15]:
data=response.json()
data.keys()

dict_keys(['instruments', 'lastTransactionID'])

In [16]:
instruments_list=data['instruments']
len(instruments_list)

129

In [17]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [18]:
key_i=['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision','marginRate']

In [19]:
#more convenient to use a dictionary , I put the name and I have the info rather than to loop into a list
instruments_dict={}

for i in instruments_list:
    key=i['name']
    instruments_dict[key]={k: i[k] for k in key_i}

In [20]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.0333'}

In [21]:
with open("D:/instruments.json","w") as f:
    f.write(json.dumps(instruments_dict, indent=2))


In [22]:
def fetch_candles (pair_name, count=10, granularity="H1"):
    url=f"{OANDA_URL}/instruments/EUR_USD/candles"
    params=dict(count=count,granularity=granularity, price="MBA")
    
    response=session.get(url, params=params, data=None, headers=None)
    data=response.json()
    if response.status_code==200:
        if 'candles' not in data:
            data=[]
        else:
            data=data['candles']
            return response.status_code, data


In [23]:



code, data=fetch_candles("EUR_USD", count=20)

In [24]:
data

[{'complete': True,
  'volume': 3413,
  'time': '2022-09-16T01:00:00.000000000Z',
  'bid': {'o': '1.00053', 'h': '1.00116', 'l': '0.99967', 'c': '0.99974'},
  'mid': {'o': '1.00060', 'h': '1.00124', 'l': '0.99975', 'c': '0.99982'},
  'ask': {'o': '1.00068', 'h': '1.00132', 'l': '0.99983', 'c': '0.99989'}},
 {'complete': True,
  'volume': 2754,
  'time': '2022-09-16T02:00:00.000000000Z',
  'bid': {'o': '0.99970', 'h': '0.99996', 'l': '0.99902', 'c': '0.99938'},
  'mid': {'o': '0.99978', 'h': '1.00004', 'l': '0.99910', 'c': '0.99945'},
  'ask': {'o': '0.99986', 'h': '1.00012', 'l': '0.99918', 'c': '0.99952'}},
 {'complete': True,
  'volume': 2337,
  'time': '2022-09-16T03:00:00.000000000Z',
  'bid': {'o': '0.99937', 'h': '1.00024', 'l': '0.99923', 'c': '0.99967'},
  'mid': {'o': '0.99945', 'h': '1.00031', 'l': '0.99930', 'c': '0.99974'},
  'ask': {'o': '0.99953', 'h': '1.00038', 'l': '0.99938', 'c': '0.99982'}},
 {'complete': True,
  'volume': 1607,
  'time': '2022-09-16T04:00:00.0000000

In [25]:
code

200

In [26]:
#time 
prices=['mid','bid','ask']
ohlc=['o','h','l','c']


In [43]:


def get_candles_df(data):
    if len(data)==0:
        return pd.DataFrame()

    prices=['mid','bid','ask']
    ohlc=list('ohlc')
    final_data=[]

    for candle in data:
        if candle['complete']==False:
            continue
        new_dict={}
        new_dict['volume']=candle['volume']
        new_dict['time']=parser.parse(candle['time'])
        for p in prices :
            for o in ohlc:
                new_dict[f"{p}_{o}"]=float(candle[p][o])
        final_data.append(new_dict)
    final_data    
    df=pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1", path="../data/"):
    code, data= fetch_candles(pair_name, count, granularity)
    if code !=200:
        print("Failed", pair_name, data)
        return
    if len(data)==0:
        print("No candles", pair_name)
    candles_df=get_candles_df(data)
    candles_df.to_pickle(f"{path}{pair_name}_{granularity}.pkl")    
    print(f"{pair_name} {granularity} {candles_df.shape[0]} {candles_df.time.min()} {candles_df.time.max()}")    

In [34]:

code, data= fetch_candles("EUR_USD", count=10, granularity="H4")

In [44]:
candles_df=get_candles_df(data)
create_data_file("EUR_USD", count=10, granularity="H4", path="D:/ML Materials/Forex Course/data/")

EUR_USD H4 10 2022-09-15 05:00:00+00:00 2022-09-16 17:00:00+00:00


In [36]:
candles_df

,volume,time,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,16947,2022-09-15 05:00:00+00:00,0.99683,0.99955,0.99558,0.99924,0.99675,0.99948,0.99551,0.99916,0.99691,0.99962,0.99565,0.99932
1,23529,2022-09-15 09:00:00+00:00,0.99923,1.00178,0.99693,1.00078,0.99916,1.00171,0.99685,1.00071,0.99930,1.00187,0.99701,1.00084
2,26454,2022-09-15 13:00:00+00:00,1.00078,1.00138,0.99815,1.00057,1.00072,1.00130,0.99807,1.00050,1.00085,1.00145,0.99822,1.00064
3,11738,2022-09-15 17:00:00+00:00,1.00058,1.00154,0.99879,0.99972,1.00051,1.00147,0.99871,0.99960,1.00065,1.00162,0.99887,0.99985
4,7086,2022-09-15 21:00:00+00:00,1.00010,1.00108,0.99862,1.00058,0.99960,1.00100,0.99853,1.00051,1.00060,1.00115,0.99870,1.00066
5,10111,2022-09-16 01:00:00+00:00,1.00060,1.00124,0.99910,0.99958,1.00053,1.00116,0.99902,0.99951,1.00068,1.00132,0.99918,0.99965
6,23329,2022-09-16 05:00:00+00:00,0.99956,0.99987,0.99452,0.99554,0.99950,0.99979,0.99444,0.99547,0.99963,0.99995,0.99459,0.99561
7,19605,2022-09-16 09:00:00+00:00,0.99552,0.99892,0.99533,0.99554,0.99545,0.99886,0.99526,0.99547,0.99558,0.99900,0.99539,0.99561
8,34670,2022-09-16 13:00:00+00:00,0.99556,1.00364,0.99537,1.00070,0.99549,1.00358,0.99530,1.00063,0.99563,1.00371,0.99544,1.00077
9,12910,2022-09-16 17:00:00+00:00,1.00072,1.00174,0.99932,1.00170,1.00065,1.00143,0.99925,1.00122,1.00079,1.00223,0.99938,1.00218


In [37]:
candles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   volume  10 non-null     int64                  
 1   time    10 non-null     datetime64[ns, tzutc()]
 2   mid_o   10 non-null     float64                
 3   mid_h   10 non-null     float64                
 4   mid_l   10 non-null     float64                
 5   mid_c   10 non-null     float64                
 6   bid_o   10 non-null     float64                
 7   bid_h   10 non-null     float64                
 8   bid_l   10 non-null     float64                
 9   bid_c   10 non-null     float64                
 10  ask_o   10 non-null     float64                
 11  ask_h   10 non-null     float64                
 12  ask_l   10 non-null     float64                
 13  ask_c   10 non-null     float64                
dtypes: datetime64[ns, tzutc()](1), float64(12), i

In [45]:
our_curr=['EUR','USD','GBP','JPY','CHF','NZD','CAD','AUD']

In [46]:
instruments_dict.keys()

dict_keys(['XCU_USD', 'WHEAT_USD', 'NZD_SGD', 'SPX500_USD', 'USD_NOK', 'UK100_GBP', 'AUD_NZD', 'XAG_USD', 'EUR_HUF', 'XAG_EUR', 'LTC_USD', 'XAG_GBP', 'NZD_USD', 'USD_CNH', 'CHF_ZAR', 'GBP_NZD', 'USB10Y_USD', 'HK33_HKD', 'USB05Y_USD', 'USB02Y_USD', 'DE10YB_EUR', 'US2000_USD', 'USB30Y_USD', 'USD_CAD', 'ZAR_JPY', 'XAG_JPY', 'ETH_USD', 'SGD_JPY', 'GBP_ZAR', 'USD_JPY', 'EUR_TRY', 'BTC_USD', 'EUR_CZK', 'EUR_JPY', 'TWIX_USD', 'AUD_SGD', 'XAG_NZD', 'WTICO_USD', 'XAG_AUD', 'EUR_NZD', 'GBP_HKD', 'CHF_JPY', 'XAU_AUD', 'EUR_HKD', 'XAU_HKD', 'XAU_JPY', 'USD_THB', 'GBP_CHF', 'CAD_JPY', 'NZD_CHF', 'ESPIX_EUR', 'AUD_HKD', 'XAG_HKD', 'USD_CHF', 'XAG_CHF', 'BCH_USD', 'AUD_USD', 'GBP_JPY', 'USD_TRY', 'NAS100_USD', 'TRY_JPY', 'USD_SGD', 'EUR_SEK', 'HKD_JPY', 'AUD_CHF', 'XAU_XAG', 'SGD_CHF', 'SOYBN_USD', 'CORN_USD', 'USD_PLN', 'GBP_AUD', 'XAU_SGD', 'AUD_JPY', 'EUR_ZAR', 'CHINAH_HKD', 'SG30_SGD', 'EUR_USD', 'EUR_CAD', 'USD_HUF', 'NL25_EUR', 'EUR_CHF', 'UK10YB_GBP', 'NZD_CAD', 'EUR_SGD', 'USD_INR', 'XAU_NZD'

In [47]:
len(instruments_dict.keys())

129

In [48]:
for p1 in our_curr:
    for p2 in our_curr:
        pr=f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr,count=4001, granularity=g, path="D:/ML Materials/Forex Course/data/")

EUR_USD H1 4001 2022-01-27 00:00:00+00:00 2022-09-16 20:00:00+00:00
EUR_USD H4 4001 2020-02-25 02:00:00+00:00 2022-09-16 17:00:00+00:00
EUR_GBP H1 4001 2022-01-27 00:00:00+00:00 2022-09-16 20:00:00+00:00
EUR_GBP H4 4001 2020-02-25 02:00:00+00:00 2022-09-16 17:00:00+00:00
EUR_JPY H1 4001 2022-01-27 00:00:00+00:00 2022-09-16 20:00:00+00:00
EUR_JPY H4 4001 2020-02-25 02:00:00+00:00 2022-09-16 17:00:00+00:00
EUR_CHF H1 4001 2022-01-27 00:00:00+00:00 2022-09-16 20:00:00+00:00
EUR_CHF H4 4001 2020-02-25 02:00:00+00:00 2022-09-16 17:00:00+00:00
EUR_NZD H1 4001 2022-01-27 00:00:00+00:00 2022-09-16 20:00:00+00:00
EUR_NZD H4 4001 2020-02-25 02:00:00+00:00 2022-09-16 17:00:00+00:00
EUR_CAD H1 4001 2022-01-27 00:00:00+00:00 2022-09-16 20:00:00+00:00
EUR_CAD H4 4001 2020-02-25 02:00:00+00:00 2022-09-16 17:00:00+00:00
EUR_AUD H1 4001 2022-01-27 00:00:00+00:00 2022-09-16 20:00:00+00:00
EUR_AUD H4 4001 2020-02-25 02:00:00+00:00 2022-09-16 17:00:00+00:00
USD_JPY H1 4001 2022-01-27 00:00:00+00:00 2022-0